In [ ]:
#!pip install transformers==4.33.1
#!pip install accelerate==0.21.0
#!pip install optimum==1.13.1
#!pip install auto-gptq==0.4.2


!pip install --upgrade git+https://github.com/huggingface/transformers
!pip install bitsandbytes
!pip install accelerate
!pip install auto-gptq
!pip install optimum
!pip install contractions

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import re

In [ ]:
script_dir = ""

In [ ]:
!ls

In [ ]:
def initiateModelAndTokenizer():

    model_name_or_path = "TheBloke/Airoboros-L2-13B-2.1-GPTQ"
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                device_map = "auto",
                                                trust_remote_code = True,
                                                revision = "main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast = True)

    return model, tokenizer

# Inicjuje model i tokenizer
model, tokenizer = initiateModelAndTokenizer()

In [ ]:
def generateDescription(quotes):

    quotes_pros = quotes.loc[quotes["sentiment"] == "positive"]
    quotes_cons = quotes.loc[quotes["sentiment"] == "negative"]
    quotes = pd.concat([quotes_pros, quotes_cons])
    quotes = quotes.reset_index(drop = True)

    user_prompt = f"""
    You will be provided with Python dataframe containing product's pros and cons (features) structured with following colums:
    category - name of a group that this feature belongs to
    quote - description of the feature extracted from user's review
    sentiment - determination if the feature is positive or negative
    Analyze which features are most frequently mentioned in provided dataframe and summarize it.
    Only describe features which are mentioned in provided dataframe.
    Do not make up information which is not mentioned in provided dataframe.
    Write it in a form of a short story.
    Do not repeat similar sentences.
    Do not use the following phrases in your description: "customer support", "packaging", "delivery", "shipping".
    Always start your description with "START" tag.
    Always end your description with "END" tag.
    Never use colons.

    This is the dataframe:

    {quotes}
    """

    prompt = f"""
    [INST] <<SYS>>
    You are an objective reviewer and your task is to summarize how other users rate the product.
    You only use information mentioned in provided data.
    You focus only on the features of the product itself.
    You do not use list structures to present data.
    You write descriptive paragraphs to explain pros and cons.
    Your description should not be longer than a few paragraphs.
    <</SYS>>
    {user_prompt}[/INST]
    """

    input_ids = tokenizer(prompt, return_tensors = "pt").input_ids.cuda()
    output = model.generate(inputs = input_ids,
                            temperature = 0.01,
                            do_sample = True,
                            top_p = 0.01,
                            top_k = 20,
                            max_new_tokens = 1024,
                            no_repeat_ngram_size = 4)

    result = tokenizer.decode(output[0])

    result = result.split("[/INST]")[1]
    print(result)
    result = re.sub("START:", "START", result)
    result = re.sub("END:", "END", result)
    result = result.split("START")[1]
    result = result.split("END")[0]
    result = result.strip()

    return result

In [ ]:

script_dir = ""

df = pd.read_csv(script_dir + "products.csv")
products = df[df['product_name'].isin(['xiaomi 13 pro', 'nokia g11', 'realme c30'])]


descriptions = []
for i in range(len(products)):
    if products.loc[i, "processed_reviews"] > 0:
        product = products.loc[i, "product_name"]
        product = re.sub(" ", "_", product)
        print(product)
        quotes = pd.read_csv(f"{script_dir}results/{product}.csv")
        description = generateDescription(quotes)
        descriptions.append({"product_name": product,
                             "description": description})

descriptions_df = pd.DataFrame(descriptions)
descriptions_df.to_csv(f"{script_dir}descriptions.csv", index = False)

nokia_8210

    
START:

The Nokia phone has received positive reviews for its performance, brand, and design. Users have praised the phone's speed and efficiency, with one reviewer stating that their mother is "very happy" with it. The brand name is also a major selling point, with reviewers noting that the phone is a "real Nokia" device. Additionally, the design of the phone has been praised for its authenticity, with reviewed mentioning that it looks like a "real" Nokia device.

END</s>
ulefone_power_armor_x11_pro

    
START:

The product's features are primarily focused on its battery life, performance, design, and software. The majority of users have praised the battery life, with one user reporting that it lasts a whole week on a single charge. The performance of the device is also commended, with no issues detected during regular use. Additionally, the design of the phone is well-received, with users finding it aesthetically pleasing and well-built.

The software, which is base